In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Questão 1

### Dataset


In [241]:
# Definindo os padrões de entrada (vértices do cubo)
entradas = np.array([
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1]
]).astype(float)

# Vetores de resposta associados a cada padrão de entrada
respostas = np.array([
    [1, -1, -1, -1, -1, -1, -1, -1],
    [-1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, 1, -1, -1, -1, -1, -1],
    [-1, -1, -1, 1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 1, -1, -1, -1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 1, -1],
    [-1, -1, -1, -1, -1, -1, -1, 1]
]).astype(float)

In [242]:
# Função para adicionar ruído aos padrões de entrada
def adicionar_ruido(entradas, raio_ruido):
    # Cria uma cópia das entradas para não alterar o array original
    entradas_ruidosas = np.copy(entradas)
    # Adiciona ruído aleatório dentro do raio especificado
    ruido = np.random.uniform(-raio_ruido, raio_ruido, entradas.shape)
    entradas_ruidosas += ruido
    return entradas_ruidosas

# Definindo o raio máximo do ruído
raio_ruido = 0.1

# Criando o conjunto de validação com ruído
conjunto_validacao = adicionar_ruido(entradas, raio_ruido)
conjunto_validacao.shape

(8, 3)

In [243]:
X = entradas.T
Y = respostas.reshape((8, len(respostas)))

print ('The shape of X: ' + str(X.shape))
print ('The shape of Y: ' + str(Y.shape))
print ('I have m = %d training examples!' % (X.shape[1]))


The shape of X: (3, 8)
The shape of Y: (8, 8)
I have m = 8 training examples!


# Perceptron de Rosenblatt

In [244]:
def layer_sizes(X, Y):
    n_x = X.shape[0]  # Size of input layer
    n_y = Y.shape[0]  # Size of output layer
    return (n_x, n_y)

In [245]:
(n_x, n_y) = layer_sizes(X, Y)
print("The size of the input layer is: n_x = " + str(n_x))
print("The size of the output layer is: n_y = " + str(n_y))

The size of the input layer is: n_x = 3
The size of the output layer is: n_y = 8


Temos 8 exemplos de treinamento, com 3 atributos cada.

In [246]:
def initialize_parameters(n_x, n_y):
    W = np.random.randn(n_y, n_x) * 0.01  # Small random weights
    b = np.zeros((n_y, 1))  # Bias initialized to zero
    return {"W": W, "b": b}

In [247]:
def forward_propagation(X, parameters):
    W = parameters['W']
    b = parameters['b']
    Z = np.dot(W, X) + b
    A = np.where(Z > 0, 1, -1)  # Activation using sign function
    return A

In [248]:
def update_parameters(parameters, X, Y, learning_rate):
    m = X.shape[1]  # Número de exemplos de treinamento
    W = parameters['W']
    b = parameters['b']
    for i in range(m):  # Iterar sobre cada exemplo de treinamento
        xi = X[:, [i]]  # Pegar a coluna i de X
        yi = Y[:, [i]]  # Pegar a coluna i de Y
        A = forward_propagation(xi, parameters)  # Previsão para o exemplo xi
        for j in range(Y.shape[0]):  # Iterar sobre cada saída
            if yi[j, 0] != A[j, 0]:  # Se a previsão estiver incorreta
                # Atualizar W e b para cada saída j
                W[j, :] = W[j, :] + learning_rate * yi[j, 0] * xi.T
                b[j, 0] = b[j, 0] + learning_rate * yi[j, 0]
    parameters['W'] = W
    parameters['b'] = b
    return parameters


In [249]:
def nn_model(X, Y, num_iterations=100, learning_rate=0.001):
    n_x, n_y = layer_sizes(X, Y)
    parameters = initialize_parameters(n_x, n_y)
    for i in range(num_iterations):
        parameters = update_parameters(parameters, X, Y, learning_rate)
    return parameters

In [250]:
parameters = nn_model(X, Y)


In [251]:
def predict(parameters, X):
    W = parameters['W']
    b = parameters['b']
    Z = np.dot(W, X) + b
    A = np.where(Z > 0, 1, -1)  # Aplica a função sinal
    
    predictions = A
    return predictions

# Realizando previsões com o modelo treinado
predictions = predict(parameters,conjunto_validacao.T)

# Agora, faça a iteração e imprima os resultados
for i in range(len(predictions)):
    # Assegure-se de que estamos comparando valores escalares
    acertou = "✅ Acertou" if np.array_equal(predictions[i], respostas[i])  else "❌ Errou"
    print(f"Entrada: {conjunto_validacao[i]} | Saída Prevista: {predictions[i]} | Esperado: {respostas[i]} | {acertou}")


Entrada: [-0.00408358  0.0168399   0.0473645 ] | Saída Prevista: [ 1 -1 -1 -1 -1 -1 -1 -1] | Esperado: [ 1. -1. -1. -1. -1. -1. -1. -1.] | ✅ Acertou
Entrada: [0.01154845 0.01730709 1.01289171] | Saída Prevista: [-1  1 -1 -1 -1 -1 -1 -1] | Esperado: [-1.  1. -1. -1. -1. -1. -1. -1.] | ✅ Acertou
Entrada: [-0.02424547  0.96748937  0.07992948] | Saída Prevista: [-1 -1  1 -1 -1 -1 -1 -1] | Esperado: [-1. -1.  1. -1. -1. -1. -1. -1.] | ✅ Acertou
Entrada: [0.02151104 0.94887063 0.99964954] | Saída Prevista: [-1 -1 -1  1 -1 -1 -1 -1] | Esperado: [-1. -1. -1.  1. -1. -1. -1. -1.] | ✅ Acertou
Entrada: [ 0.9660697   0.08673836 -0.09849313] | Saída Prevista: [-1 -1 -1 -1  1 -1 -1 -1] | Esperado: [-1. -1. -1. -1.  1. -1. -1. -1.] | ✅ Acertou
Entrada: [ 0.94506656 -0.02692864  0.99756196] | Saída Prevista: [-1 -1 -1 -1 -1  1 -1 -1] | Esperado: [-1. -1. -1. -1. -1.  1. -1. -1.] | ✅ Acertou
Entrada: [1.0701635  0.91757752 0.06117298] | Saída Prevista: [-1 -1 -1 -1 -1 -1  1 -1] | Esperado: [-1. -1. -1.

# Questão 2

In [179]:
# Definindo a função lógica XOR
def xor_function(x1, x2):
    return np.where(x1 == x2, 0, 1)

# Gerando um conjunto de amostras para a função XOR
np.random.seed(42)  # Para resultados reprodutíveis
x1_samples = np.random.randint(0, 2, 100)  # Gerando 100 amostras para x1
x2_samples = np.random.randint(0, 2, 100)  # Gerando 100 amostras para x2
y_samples = xor_function(x1_samples, x2_samples)  # Calculando os rótulos com a função XOR

# Dividindo as amostras em conjuntos de treinamento (70%) e validação (30%)
indices = np.random.permutation(len(x1_samples))
train_indices = indices[:70]
val_indices = indices[70:]

x_train = np.array([x1_samples[train_indices], x2_samples[train_indices]]).T
y_train = y_samples[train_indices]
x_val = np.array([x1_samples[val_indices], x2_samples[val_indices]]).T
y_val = y_samples[val_indices]

# Verificando as amostras de treinamento e validação
x_train, y_train, x_val, y_val

(array([[1, 0],
        [0, 1],
        [1, 0],
        [0, 0],
        [0, 1],
        [0, 1],
        [1, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 0],
        [1, 1],
        [1, 1],
        [1, 1],
        [0, 0],
        [1, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 1],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [0, 1],
        [0, 1],
        [1, 1],
        [0, 1],
        [0, 0],
        [0, 0],
        [1, 1],
        [1, 0],
        [1, 1],
        [1, 0],
        [0, 1],
        [1, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 1],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [1, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 1],
        [0, 1],
        

In [ ]:
def initialize_parameters_deep(layer_dims):

    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
    
        parameters['W' + str(l)] =  np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

        
    return parameters

In [ ]:
def linear_forward(A, W, b):
  
    Z = np.dot(W,A) + b
    cache = (A, W, b)
    
    return Z, cache

In [ ]:
def L_model_forward(X, parameters):
    
    caches = []
    A = X
    L = len(parameters) // 2  # number of layers in the neural network
    
    for l in range(1, L):
        A_prev = A
        
        # Camadas ocultas
        A, cache = linear_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)])
        caches.append(cache)

    # Ultima camada 
    AL, cache = linear_forward(A, parameters['W' + str(L)], parameters['b' + str(L)])
    caches.append(cache)
          
    return AL, caches

In [ ]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]
    cost = -np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL))/ m
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).

    return cost

In [ ]:
def nn_model(X, Y, num_iterations=10, print_cost=False, learning_rate = 0.01):
    
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[1]
    
    # Initialize parameters
    parameters = initialize_parameters(n_x,n_y)
    
    # Loop
    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters, n_y". Outputs: "Y_hat".
        Y_hat = forward_propagation(X,parameters)
        
        # Cost function. Inputs: "Y_hat, Y". Outputs: "cost".
        cost = compute_cost(Y_hat,Y)

        # Parameters update:
        # Backward propagation. Inputs: "parameters, X, Y, Y_hat". Outputs: "grads".
        dW = (1/X.shape[1]) * np.dot((Y_hat - Y), X.T)
        db = (1/X.shape[1]) * np.sum(Y_hat - Y, axis=1, keepdims=True)

        # Update rule for each parameter.
        parameters["W"] = parameters["W"] - learning_rate * dW
        parameters["b"] = parameters["b"] - learning_rate * db

    return parameters

In [ ]:
def L_model_backward(AL, Y, caches):
   
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    

    return grads

In [ ]:
import copy

def update_parameters(params, grads, learning_rate):
   
    parameters = copy.deepcopy(params)
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
       
    return parameters